#### Lo primero es crear la base de datos. Yo lo he hecho en Workbench de forma manual. También podría haberlo hecho por aquí, pero bueno, no es complicado. A partir de aquí importamos librerías y hacemos conexión con la base de datos que yo he llamado futbol_data

In [5]:
import mysql.connector
import pandas as pd

Primero, establece la conexión con tu base de datos. Asegúrate de cambiar las credenciales y el nombre de la base de datos según tus necesidades:

In [6]:
conexion = mysql.connector.connect(
    host='localhost',
    user='root',  # Asegúrate de cambiar 'root' por tu usuario de MySQL si es diferente
    password='rubenico',  # Usa tu contraseña real de MySQL aquí
    database="futbol_data"  # Cambia "futbol_data" por el nombre real de tu base de datos si es diferente
)

#### Definir la Función para Ejecutar Consultas
Ahora, define la función execute_query que toma una consulta SQL como argumento, ejecuta la consulta y devuelve los resultados en un DataFrame de pandas:

In [13]:
def execute_query(query):
    cursor = conexion.cursor()
    cursor.execute(query)
    try:
        # Solo intenta obtener los resultados si hay una descripción del cursor (es decir, si hay resultados)
        columnas = [desc[0] for desc in cursor.description]  # Obtiene los nombres de las columnas
        resultados = cursor.fetchall()  # Obtiene todos los resultados de la consulta
        df = pd.DataFrame(resultados, columns=columnas)  # Crea un DataFrame con los resultados
        cursor.close()  # Cierra el cursor
        return df.head(50)  # Retorna las primeras 50 filas del DataFrame
    except TypeError:
        # Maneja el caso de que no hay resultados para devolver (como en CREATE TABLE)
        conexion.commit()  # Asegúrate de hacer commit de la transacción
        cursor.close()  # Cierra el cursor
        print("Query executed successfully")
        return None



##### Imagen de lo que queremos construir 
<img src="imagen_base_datos.jpg"/>

* Lo primero que vamos a hacer es cargar todos los csv para ver las columnas de las que se componen para poder montar las tablas de la base de datos

In [9]:
# Asumiendo que tenemos los archivos csv en la carpeta 'datos'
df_competitions = pd.read_csv('datos/competitions.csv')
df_clubs = pd.read_csv('datos/clubs.csv')
df_games = pd.read_csv('datos/games.csv')
df_appearances = pd.read_csv('datos/appearances.csv')
df_club_games = pd.read_csv('datos/club_games.csv')
df_game_events = pd.read_csv('datos/game_events.csv')
df_game_lineups = pd.read_csv('datos/game_lineups.csv')
df_player_valuations = pd.read_csv('datos/player_valuations.csv')
df_players = pd.read_csv('datos/players.csv')


# Mostrar las columnas del DataFrame
print(df_competitions.columns.tolist())
print(df_clubs.columns.tolist())
print(df_games.columns.tolist())
print(df_appearances.columns.tolist())
print(df_club_games.columns.tolist())
print(df_game_events.columns.tolist())
print(df_game_lineups.columns.tolist())
print(df_player_valuations.columns.tolist())
print(df_players.columns.tolist())

['competition_id', 'competition_code', 'name', 'sub_type', 'type', 'country_id', 'country_name', 'domestic_league_code', 'confederation', 'url']
['club_id', 'club_code', 'name', 'domestic_competition_id', 'total_market_value', 'squad_size', 'average_age', 'foreigners_number', 'foreigners_percentage', 'national_team_players', 'stadium_name', 'stadium_seats', 'net_transfer_record', 'coach_name', 'last_season', 'filename', 'url']
['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id', 'away_club_id', 'home_club_goals', 'away_club_goals', 'home_club_position', 'away_club_position', 'home_club_manager_name', 'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url', 'home_club_formation', 'away_club_formation', 'home_club_name', 'away_club_name', 'aggregate', 'competition_type']
['appearance_id', 'game_id', 'player_id', 'player_club_id', 'player_current_club_id', 'date', 'player_name', 'competition_id', 'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_pla

### Ahora vamos a crear las tablas en la base de datos

In [14]:
query_competitions = """
CREATE TABLE IF NOT EXISTS competitions (
    competition_id INT PRIMARY KEY,
    competition_code VARCHAR(10),
    name VARCHAR(255) NOT NULL,
    sub_type VARCHAR(50),
    type VARCHAR(50),
    country_id INT,
    country_name VARCHAR(100),
    domestic_league_code VARCHAR(10),
    confederation VARCHAR(50),
    url VARCHAR(255)
);
"""
execute_query(query_competitions)

Query executed successfully


In [15]:
query_clubs = """
CREATE TABLE IF NOT EXISTS clubs (
    club_id INT PRIMARY KEY,
    club_code VARCHAR(10),
    name VARCHAR(255) NOT NULL,
    domestic_competition_id INT,
    total_market_value FLOAT,
    squad_size INT,
    average_age FLOAT,
    foreigners_number INT,
    foreigners_percentage FLOAT,
    national_team_players INT,
    stadium_name VARCHAR(255),
    stadium_seats INT,
    net_transfer_record VARCHAR(255),
    coach_name VARCHAR(255),
    last_season VARCHAR(50),
    filename VARCHAR(100),
    url VARCHAR(255),
    FOREIGN KEY (domestic_competition_id) REFERENCES competitions(competition_id)
);
"""
execute_query(query_clubs)


Query executed successfully


In [16]:
query_players = """
CREATE TABLE IF NOT EXISTS players (
    player_id INT PRIMARY KEY,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    name VARCHAR(255),
    last_season VARCHAR(50),
    current_club_id INT,
    player_code VARCHAR(50),
    country_of_birth VARCHAR(255),
    city_of_birth VARCHAR(255),
    country_of_citizenship VARCHAR(255),
    date_of_birth DATE,
    sub_position VARCHAR(50),
    position VARCHAR(50),
    foot VARCHAR(50),
    height_in_cm INT,
    contract_expiration_date DATE,
    agent_name VARCHAR(255),
    image_url VARCHAR(255),
    url VARCHAR(255),
    current_club_domestic_competition_id INT,
    current_club_name VARCHAR(255),
    market_value_in_eur FLOAT,
    highest_market_value_in_eur FLOAT,
    FOREIGN KEY (current_club_id) REFERENCES clubs(club_id)
);
"""
execute_query(query_players)


Query executed successfully


In [17]:
query_games = """
CREATE TABLE IF NOT EXISTS games (
    game_id INT PRIMARY KEY,
    competition_id INT,
    season VARCHAR(10),
    round VARCHAR(50),
    date DATE,
    home_club_id INT,
    away_club_id INT,
    home_club_goals INT,
    away_club_goals INT,
    home_club_position INT,
    away_club_position INT,
    home_club_manager_name VARCHAR(255),
    away_club_manager_name VARCHAR(255),
    stadium VARCHAR(255),
    attendance INT,
    referee VARCHAR(255),
    url VARCHAR(255),
    home_club_formation VARCHAR(50),
    away_club_formation VARCHAR(50),
    home_club_name VARCHAR(255),
    away_club_name VARCHAR(255),
    aggregate INT,
    competition_type VARCHAR(50),
    FOREIGN KEY (competition_id) REFERENCES competitions(competition_id),
    FOREIGN KEY (home_club_id) REFERENCES clubs(club_id),
    FOREIGN KEY (away_club_id) REFERENCES clubs(club_id)
);
"""
execute_query(query_games)


Query executed successfully


In [18]:
query_club_games = """
CREATE TABLE IF NOT EXISTS club_games (
    game_id INT,
    club_id INT,
    own_goals INT,
    own_position INT,
    own_manager_name VARCHAR(255),
    opponent_id INT,
    opponent_goals INT,
    opponent_position INT,
    opponent_manager_name VARCHAR(255),
    hosting BOOLEAN,
    is_win BOOLEAN,
    PRIMARY KEY (game_id, club_id),
    FOREIGN KEY (game_id) REFERENCES games(game_id),
    FOREIGN KEY (club_id) REFERENCES clubs(club_id),
    FOREIGN KEY (opponent_id) REFERENCES clubs(club_id)
);
"""
execute_query(query_club_games)


Query executed successfully


In [19]:
query_appearances = """
CREATE TABLE IF NOT EXISTS appearances (
    appearance_id INT PRIMARY KEY,
    game_id INT,
    player_id INT,
    player_club_id INT,
    player_current_club_id INT,
    date DATE,
    player_name VARCHAR(255),
    competition_id INT,
    yellow_cards INT,
    red_cards INT,
    goals INT,
    assists INT,
    minutes_played INT,
    FOREIGN KEY (game_id) REFERENCES games(game_id),
    FOREIGN KEY (player_id) REFERENCES players(player_id),
    FOREIGN KEY (competition_id) REFERENCES competitions(competition_id),
    FOREIGN KEY (player_club_id) REFERENCES clubs(club_id)
);
"""
execute_query(query_appearances)


Query executed successfully


In [20]:
query_game_lineups = """
CREATE TABLE IF NOT EXISTS game_lineups (
    game_lineups_id INT PRIMARY KEY,
    game_id INT,
    club_id INT,
    type VARCHAR(255),
    number INT,
    player_id INT,
    player_name VARCHAR(255),
    team_captain BOOLEAN,
    position VARCHAR(50),
    FOREIGN KEY (game_id) REFERENCES games(game_id),
    FOREIGN KEY (club_id) REFERENCES clubs(club_id),
    FOREIGN KEY (player_id) REFERENCES players(player_id)
);
"""
execute_query(query_game_lineups)


Query executed successfully


In [21]:
query_player_valuations = """
CREATE TABLE IF NOT EXISTS player_valuations (
    player_id INT,
    date DATE,
    market_value_in_eur FLOAT,
    current_club_id INT,
    player_club_domestic_competition_id INT,
    PRIMARY KEY (player_id, date),
    FOREIGN KEY (player_id) REFERENCES players(player_id),
    FOREIGN KEY (current_club_id) REFERENCES clubs(club_id)
);
"""
execute_query(query_player_valuations)


Query executed successfully


In [22]:
query_game_events = """
CREATE TABLE IF NOT EXISTS game_events (
    game_event_id INT PRIMARY KEY,
    date DATE,
    game_id INT,
    minute INT,
    type VARCHAR(255),
    club_id INT,
    player_id INT,
    description TEXT,
    player_in_id INT,
    player_out_id INT,
    player_assist_id INT,
    FOREIGN KEY (game_id) REFERENCES games(game_id),
    FOREIGN KEY (club_id) REFERENCES clubs(club_id),
    FOREIGN KEY (player_id) REFERENCES players(player_id),
    FOREIGN KEY (player_in_id) REFERENCES players(player_id),
    FOREIGN KEY (player_out_id) REFERENCES players(player_id),
    FOREIGN KEY (player_assist_id) REFERENCES players(player_id)
);
"""
execute_query(query_game_events)


Query executed successfully


* Ya tenemos las tablas creadas en la base de datos con sus claves. Ahora podemos hacer una comprobación para ver que es así y sus primary keys para comprobar que todo está ok.

In [24]:
def show_tables_with_primary_keys():
    cursor = conexion.cursor()
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    
    table_primary_keys = {}
    
    for table in tables:
        table_name = table[0]
        cursor.execute(f"SHOW KEYS FROM `{table_name}` WHERE Key_name = 'PRIMARY'")
        primary_keys = cursor.fetchall()
        # Cada fila en primary_keys contiene información sobre una columna de clave primaria
        # El nombre de la columna está en la posición 4 de la fila
        pk_column_names = [pk[4] for pk in primary_keys]
        table_primary_keys[table_name] = pk_column_names

    cursor.close()
    return table_primary_keys

# Llamamos a la función y mostramos las tablas con sus claves primarias
tables_with_pks = show_tables_with_primary_keys()
for table, pks in tables_with_pks.items():
    print(f"Tabla: {table}, Clave(s) Primaria(s): {', '.join(pks)}")



Tabla: appearances, Clave(s) Primaria(s): appearance_id
Tabla: club_games, Clave(s) Primaria(s): game_id, club_id
Tabla: clubs, Clave(s) Primaria(s): club_id
Tabla: competitions, Clave(s) Primaria(s): competition_id
Tabla: game_events, Clave(s) Primaria(s): game_event_id
Tabla: game_lineups, Clave(s) Primaria(s): game_lineups_id
Tabla: games, Clave(s) Primaria(s): game_id
Tabla: player_valuations, Clave(s) Primaria(s): player_id, date
Tabla: players, Clave(s) Primaria(s): player_id


* Ahora lo que vamos a hacer es meter los datos. Para ello como lo tenemos todos los csv guardados en la carpeta datos, vamos a preparar una función para cada uno de ellos.


In [33]:
def drop_foreign_key_constraints():
    cursor = conexion.cursor()
    try:
        cursor.execute("ALTER TABLE appearances DROP FOREIGN KEY appearances_ibfk_3;")
        conexion.commit()
        print("Foreign key constraints dropped successfully.")
    except Exception as e:
        print(f"Error dropping foreign key constraints: {e}")
        conexion.rollback()
    finally:
        cursor.close()

drop_foreign_key_constraints()



Foreign key constraints dropped successfully.


In [34]:
def modify_competition_id_column():
    cursor = conexion.cursor()
    try:
        cursor.execute("ALTER TABLE competitions MODIFY competition_id VARCHAR(255);")
        cursor.execute("ALTER TABLE appearances MODIFY competition_id VARCHAR(255);")  # Asegúrate de que 'appearances' tenga esta columna para modificar
        conexion.commit()
        print("Column 'competition_id' modified successfully in all affected tables.")
    except Exception as e:
        print(f"Error modifying columns: {e}")
        conexion.rollback()
    finally:
        cursor.close()

modify_competition_id_column()


Column 'competition_id' modified successfully in all affected tables.


In [35]:
def add_foreign_key_constraints():
    cursor = conexion.cursor()
    try:
        cursor.execute("""
            ALTER TABLE games
            ADD CONSTRAINT games_ibfk_1 FOREIGN KEY (competition_id) REFERENCES competitions(competition_id);
        """)
        cursor.execute("""
            ALTER TABLE appearances
            ADD CONSTRAINT appearances_ibfk_3 FOREIGN KEY (competition_id) REFERENCES competitions(competition_id);
        """)
        conexion.commit()
        print("Foreign key constraints re-established successfully.")
    except Exception as e:
        print(f"Error adding foreign key constraints: {e}")
        conexion.rollback()
    finally:
        cursor.close()

add_foreign_key_constraints()


Error adding foreign key constraints: 3780 (HY000): Referencing column 'competition_id' and referenced column 'competition_id' in foreign key constraint 'games_ibfk_1' are incompatible.


In [28]:

def insert_competitions(df):
    cursor = conexion.cursor()
    for index, row in df.iterrows():
        sql = """
        INSERT INTO competitions (competition_id, competition_code, name, sub_type, type, country_id, country_name, domestic_league_code, confederation, url)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = (row['competition_id'], row['competition_code'], row['name'], row['sub_type'], row['type'], row['country_id'], row['country_name'], row['domestic_league_code'], row['confederation'], row['url'])
        try:
            cursor.execute(sql, values)
            conexion.commit()
        except Exception as e:
            print(f"Error inserting row {index}: {e}")
            conexion.rollback()
    cursor.close()


insert_competitions(df_competitions)

Error inserting row 0: 1366 (HY000): Incorrect integer value: 'CIT' for column 'competition_id' at row 1
Error inserting row 1: 1366 (HY000): Incorrect integer value: 'NLSC' for column 'competition_id' at row 1
Error inserting row 2: 1366 (HY000): Incorrect integer value: 'GRP' for column 'competition_id' at row 1
Error inserting row 3: 1366 (HY000): Incorrect integer value: 'POSU' for column 'competition_id' at row 1
Error inserting row 4: 1366 (HY000): Incorrect integer value: 'RUSS' for column 'competition_id' at row 1
Error inserting row 5: 1366 (HY000): Incorrect integer value: 'SUC' for column 'competition_id' at row 1
Error inserting row 6: 1366 (HY000): Incorrect integer value: 'USC' for column 'competition_id' at row 1
Error inserting row 7: 1366 (HY000): Incorrect integer value: 'DK1' for column 'competition_id' at row 1
Error inserting row 8: 1366 (HY000): Incorrect integer value: 'EL' for column 'competition_id' at row 1
Error inserting row 9: 1366 (HY000): Incorrect intege